In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [2]:
print(X[0])

[[[0.31764706]
  [0.36862745]
  [0.32941176]
  ...
  [0.14901961]
  [0.23137255]
  [0.21176471]]

 [[0.36862745]
  [0.29019608]
  [0.29803922]
  ...
  [0.16470588]
  [0.31372549]
  [0.30196078]]

 [[0.30980392]
  [0.35686275]
  [0.30196078]
  ...
  [0.14901961]
  [0.25490196]
  [0.36470588]]

 ...

 [[0.63921569]
  [0.6       ]
  [0.65098039]
  ...
  [0.34509804]
  [0.31372549]
  [0.17647059]]

 [[0.63921569]
  [0.62352941]
  [0.67058824]
  ...
  [0.36862745]
  [0.34509804]
  [0.32156863]]

 [[0.60392157]
  [0.60784314]
  [0.57254902]
  ...
  [0.36862745]
  [0.39215686]
  [0.39607843]]]


In [3]:
print(X.shape)

(24946, 50, 50, 1)


In [7]:
import numpy as np

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

y = np.array(y)

model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3)

Train on 17462 samples, validate on 7484 samples
Epoch 1/3
17462/17462 [==============================] - 1512s 87ms/sample - loss: 0.6457 - accuracy: 0.6318 - val_loss: 0.5911 - val_accuracy: 0.6904
Epoch 2/3
17462/17462 [==============================] - 1517s 87ms/sample - loss: 0.5747 - accuracy: 0.7084 - val_loss: 0.5325 - val_accuracy: 0.7441
Epoch 3/3
17462/17462 [==============================] - 1590s 91ms/sample - loss: 0.5251 - accuracy: 0.7441 - val_loss: 0.5167 - val_accuracy: 0.7453


In [8]:
model.save('CNN_model.h5')   # HDF5 file, you have to pip3 install h5py if don't have it

In [9]:
from tensorflow.keras.callbacks import TensorBoard

In [10]:
NAME = "Cats-vs-dogs-CNN"

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [16]:
import pickle
import tensorflow as tf
from tensorflow import keras
import time

import os
PATH = os.path.join('logs', NAME)
tensorboard = TensorBoard(log_dir=PATH)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 48, 256)       2560      
_________________________________________________________________
activation_6 (Activation)    (None, 48, 48, 256)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 256)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 256)       590080    
_________________________________________________________________
activation_7 (Activation)    (None, 22, 22, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 30976)            

In [19]:
from keras.models import Model
from keras.initializers import Ones, Zeros

class Average_Saliency(object):
    def __init__(self, model, output_index=0):
        pass

    def get_grad(self, input_image):
        pass

    def get_average_grad(self, input_image, stdev_spread=.2, nsamples=50):
        stdev = stdev_spread * (np.max(input_image) - np.min(input_image))

        total_gradients = np.zeros_like(input_image, dtype = np.float64)
        for i in range(nsamples):
            noise = np.random.normal(0, stdev, input_image.shape)
            x_value_plus_noise = input_image + noise

            total_gradients += self.get_mask(x_value_plus_noise)

        return total_gradients / nsamples

class Single_Saliency(Average_Saliency):

    def __init__(self, model, output_index = 0):
        # Define the function to compute the gradient
        input_tensors = [model.input]
        gradients = model.optimizer.get_gradients(model.output[0][output_index], model.input)
        self.compute_gradients = K.function(inputs = input_tensors, outputs = gradients)

    def get_grad(self, input_image):
        # Execute the function to compute the gradient
        x_value = np.expand_dims(input_image, axis=0)
        gradients = self.compute_gradients([x_value])[0][0]

        return gradients

Using TensorFlow backend.


In [21]:
# import matplotlib.pyplot as plt
import numpy as np

New_X = X/255

img = New_X[0]

single = Single_Saliency(model, 10)
grad = single.get_grad(img)
filter_grad = (grad > 0.0).reshape((50, 50))
average_grad = single.get_average_grad(img)
filter_average_grad = (average_grad > 0.0).reshape((50, 50))

plt.imshow(img.reshape((50, 50)), cmap = 'gray')
plt.show()


ValueError: slice index 10 of dimension 0 out of bounds. for 'strided_slice_3' (op: 'StridedSlice') with input shapes: [1], [1], [1], [1] and with computed input tensors: input[1] = <10>, input[2] = <11>, input[3] = <1>.

In [ ]:
plt.imshow(grad.reshape((50, 50)), cmap = 'jet')
plt.show()

plt.imshow(grad.reshape((50, 50)) * filter_grad, cmap = 'gray')
plt.show()
plt.imshow(average_grad.reshape((50, 50)), cmap = 'jet')
plt.show()

plt.imshow(average_grad.reshape((50, 50)) * filter_average_grad, cmap = 'gray')

plt.show()